In [45]:
import numpy as np
import cv2
import napari
import time
import os

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from aicsimageio.readers.bioformats_reader import BioformatsReader
from aicsimageio.readers.czi_reader import CziReader

# function for core algorithm
from aicssegmentation.core.seg_dot import dot_3d, dot_3d_wrapper 
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice, image_smoothing_gaussian_3d, edge_preserving_smoothing_3d
from skimage.morphology import remove_small_objects, dilation, erosion, ball     # function for post-processing (size filter)
from skimage.segmentation import watershed, clear_border
from skimage.feature import peak_local_max
from skimage.measure import label
from skimage.io import imsave
from scipy.ndimage import distance_transform_edt, binary_fill_holes

In [50]:
## Load the data

#####################
# User inputs the file path
filePath = "//research.files.med.Harvard.edu/SysBio/MEGASON LAB/People/RikkiGarner/20220922_h2bRRvT_12hpf_LSM980/20220922_h2bRRvT_11hpf_20X-03-Airyscan Processing-01.czi"
#####################

# Load the image using the AICS importer
img_from_native = AICSImage(filePath, reader=CziReader)

# Load the scale of each axis
micronsPerPixel = [img_from_native.physical_pixel_sizes.Z, img_from_native.physical_pixel_sizes.X , img_from_native.physical_pixel_sizes.Y]
print(micronsPerPixel)

# Load in the first image
raw_img = img_from_native.get_image_dask_data("TCZYX")

# Print the size of the object in the format (#timepoints, #channels, #pixelsZ, #pixelsY, #pixelsX)
print(raw_img.shape)

## Visualize the raw images

# Remove the old image if it exists
if 'viewer' in locals():
    try:
        viewer.close()
    except:
        del viewer
        

viewer.scale_bar
viewer.axes.visible = True
viewer.axes.colored = False
viewer.scale_bar.visible = True
viewer.scale_bar.unit = "um"  
viewer.scale_bar.ticks = False





[1.5, 0.59188755580357, 0.59188755580357]
(247, 1, 121, 1000, 1000)


In [40]:

## Save the 3D volume rendering as a movie

# Make a folder to save the movie
# Create the folder name
folderName = filePath.rsplit('/', 1)[-1].rsplit('.', 1)[0]+'_Volume'
# Add this folder to the existing path of the data file
folderPath = filePath.rsplit('/', 1)[0]+'/'+folderName+'/'
# Make the directory, if it doesn't exist
if ~os.path.isdir(folderPath):
    os.mkdir(folderPath)
# Make the file path prefic
filePathPrefix = folderPath+folderName

# Set the timepoint to zero
dimensionState = viewer.dims.current_step
dimensionState = list(dimensionState)
dimensionState[0] = 0
dimensionState = tuple(dimensionState)
viewer.dims.current_step = dimensionState

# Update the filepath
filePath = filePathPrefix+'_'+str(dimensionState[0])+'.tif'

# If the file already exists, delete it
if os.path.isfile(filePath):
    os.remove(filePath)    

# Take a screenshot
screenshot = viewer.screenshot()

# Save the screenshot
imsave(filePath, screenshot)

for tp in range(0, raw_img.shape[0]+1):
    # Update the time step
    dimensionState = viewer.dims.current_step
    dimensionState = list(dimensionState)
    dimensionState[0] = dimensionState[0] + 1
    dimensionState = tuple(dimensionState)
    viewer.dims.current_step = dimensionState
    
    # Update the filepath
    filePath = filePathPrefix+'_'+str(dimensionState[0])+'.tif'
    
    if os.path.isfile(filePath):
        os.remove(filePath)    

    # Take a screenshot
    screenshot = viewer.screenshot()

    # Save the screenshot
    imsave(filePath, screenshot)

(2, 60, 500, 500)
